# Align_HE_DL_ROI_CNT
## This workflow reads HE wsi, DLmask, ROImask, nuclear contour
## crop, rotate, and crop each one of them and save as new images

In [1]:
import pandas as pd
from PIL import Image
Image.MAX_IMAGE_PIXELS=None
import numpy as np
from skimage.morphology import closing, square, remove_small_objects, remove_small_holes
from math import atan2, degrees
import os
import cv2
from openslide import OpenSlide

In [2]:
def rotate_image_cv2(mat, angle):
    height, width = mat.shape[:2] # image shape has 3 dimensions
    image_center = (width/2, height/2) # getRotationMatrix2D needs coordinates in reverse order (width, height) compared to shape
    rotation_mat = cv2.getRotationMatrix2D(image_center, angle, 1.)
    # rotation calculates the cos and sin, taking absolutes of those.
    abs_cos = abs(rotation_mat[0,0])
    abs_sin = abs(rotation_mat[0,1])
    # find the new width and height bounds
    bound_w = int(height * abs_sin + width * abs_cos)
    bound_h = int(height * abs_cos + width * abs_sin)
    # subtract old image center (bringing image back to origo) and adding the new image center coordinates
    rotation_mat[0, 2] += bound_w/2 - image_center[0]
    rotation_mat[1, 2] += bound_h/2 - image_center[1]
    # rotate image with the new bounds and translated rotation matrix
    rotated_mat = cv2.warpAffine(mat, rotation_mat, (bound_w, bound_h), flags=cv2.INTER_NEAREST)
    return rotated_mat

In [3]:
realjsonsrc = r'\\fatherserverdw\kyuex\clue images\hovernet_out\json'
roisrc = r'\\fatherserverdw\kyuex\clue images\annotations\roi\labeledmask_20rsf'
ndpisrc = r'\\fatherserverdw\kyuex\clue images'
dlsrc = r'\\fatherserverdw\kyuex\clue images\1um\classification_v9_combined'

dst = r'\\fatherserverdw\kyuex\datadst\20220929'
imcropdst =os.path.join(dst,'imcrop')
dlcropdst = os.path.join(dst,'dlcrop')
nuccropdst = os.path.join(dst,'nuccrop')
if not os.path.exists(dst): os.mkdir(dst)
if not os.path.exists(imcropdst): os.mkdir(imcropdst)
if not os.path.exists(dlcropdst): os.mkdir(dlcropdst)
if not os.path.exists(nuccropdst): os.mkdir(nuccropdst)

ClUedegrot_df = pd.read_excel(r"\\fatherserverdw\kyuex\datadst\WSI_rotation_crop_LUT.xlsx")
ClUedegrot_df = ClUedegrot_df.astype("object")

In [4]:
index = 10
row = ClUedegrot_df.iloc[index]

fn = os.path.splitext(row['filename'])[0]

mask = Image.open(os.path.join(dlsrc,'{}.{}'.format(fn,'tif')))
ndpi = OpenSlide(os.path.join(ndpisrc,'{}.{}'.format(fn,'ndpi')))
jsdst = os.path.join(realjsonsrc,'{}.{}'.format(fn,'json'))
json = pd.read_json(jsdst, orient='index')
roi= Image.open(os.path.join(roisrc,'{}.{}'.format(fn,'png'))) #roi is very small

In [7]:
[x,y] = roi.size
(w,h) = ndpi.level_dimensions[0]
rsf = [w/x,h/y]
rsf = rsf[0]

In [8]:
json = pd.DataFrame(json[0].loc['nuc']).T.drop(columns=['type_prob'])
json = json[json['contour'].map(len) > 5].reset_index(drop=True)

In [9]:
roiarr = np.array(roi)
def isinroi(row):
    newrow = [round(_/16) for _ in row]
    return roiarr[newrow[1],newrow[0]]
json['inroi'] =  json['centroid'].apply(lambda row: isinroi(row))
jsoninroi = json[json['inroi']>0]

In [10]:
#create binary labeled mask of nuclei inside of roi
nuc_image = np.zeros((h,w), dtype=np.int32) #need to flip h and w
for idx,ct in enumerate(jsoninroi['contour']):
     cv2.fillPoly(nuc_image, pts=[np.array(ct).astype(np.int32)],  color=idx+1)

In [11]:
#create DLmask inside of roi
mask_resized = mask.resize(roi.size, resample=0) #nearest interpolation to rescale
DLsmall = np.array(mask_resized)
DLinroi = np.multiply(DLsmall,roiarr>0)

In [12]:
#create H&E
targetlevel = ndpi.get_best_level_for_downsample(rsf)

In [13]:
HE = ndpi.read_region(location=(0,0),level=targetlevel,size=ndpi.level_dimensions[targetlevel])

In [14]:
minDermhole = 5000
minepisize = 1000
whitespace=12

epi = (DLinroi == 1) | (DLinroi == 2)
derm = (2 < DLinroi) & (DLinroi < whitespace)
derm = remove_small_holes(derm, area_threshold=minDermhole)

epi2 = epi & ~derm
epi2 = remove_small_objects(epi2, min_size=minepisize, connectivity=2)
numsecmax = np.max(roiarr)

In [15]:
for numsec in range(1,numsecmax):
    print('section N: ', numsec, '/', numsecmax)
    roitmp = roiarr == numsec #roitmp is logical, not greyscale
    mskepi = roitmp & epi2
    DLtmp = np.multiply(DLinroi,roitmp) #roiscale
    HEtmp = np.multiply(np.array(HE)[:,:,:3],np.repeat(roitmp[:,:,np.newaxis],3,axis=2)) #roiscale

    # align horizontal
    [xt0, yt0] = np.where(mskepi)
    vertices = np.array([xt0[::10], yt0[::10]]).T
    vc = vertices - vertices.mean(axis=0)

    U, S, Vt = np.linalg.svd(vc)
    k = Vt.T
    d0 = degrees(atan2(k[1, 1], k[1, 0]))
    if np.linalg.det(k)<0: d0=-d0
    if d0 < 0: d0 = d0 + 360
    #clear variables, we just need d0
    del vertices, vc, U, S, Vt,k,mskepi

    #enlarge DLtmp from roiscale to 20x wsi scale
    DLtmplarge = cv2.resize(DLtmp.astype(np.uint8), dsize=(w,h), interpolation=cv2.INTER_NEAREST)
    [xt, yt] = np.where(DLtmplarge[:,:])
    bbox = [np.min(xt),np.max(xt),np.min(yt),np.max(yt)]
    bbox = [round(_) for _ in bbox]
    DLrot = DLtmplarge[bbox[0]:bbox[1],bbox[2]:bbox[3]] #crop to save memory for practical rotation
    DLrot = rotate_image_cv2(DLrot, d0) #rotate

    #check if section is upside-down
    [xderm, yderm] = np.where(DLrot)
    [xepi, yepi] = np.where((DLrot == 1) | (DLrot == 2))
    if np.mean(xderm) - np.mean(xepi) < 0: #if dermis is above epidermis, then flip
        DLrot = np.rot90(np.rot90(DLrot))
        d0 += 180

    #crop again to remove border effect
    [xt2, yt2] = np.where(DLrot)
    bbox2 = [np.min(xt2),np.max(xt2),np.min(yt2),np.max(yt2)]
    bbox2 = [round(_) for _ in bbox2]
    DLrot = DLrot[bbox2[0]:bbox2[1],bbox2[2]:bbox2[3]]

    #crop nuclei labeled binary image using roi/DLtmplarge
    nuctmp = nuc_image[bbox[0]:bbox[1],bbox[2]:bbox[3]] #crop
    nucrot = rotate_image_cv2(nuctmp, d0)                           #rotate
    nucrot = nucrot[bbox2[0]:bbox2[1],bbox2[2]:bbox2[3]] #crop

    #crop HE
    bboxsmall = [round(_/rsf) for _ in bbox]
    bboxsmall2 = [round(_/rsf) for _ in bbox2]
    HEtmp = HEtmp[bboxsmall[0]:bboxsmall[1],bboxsmall[2]:bboxsmall[3]] #crop
    HErot = rotate_image_cv2(HEtmp, d0)                           #rotate
    HErot = HErot[bboxsmall2[0]:bboxsmall2[1],bboxsmall2[2]:bboxsmall2[3]] #crop
    HErot[HErot == 0] = 235
    HEbig = cv2.resize(HErot.astype(np.uint8), dsize=[_ for _ in DLrot.shape][::-1], interpolation=cv2.INTER_NEAREST)

    dstfn = fn+'sec{}'.format(numsec)+'.png'

    #save images
    Image.fromarray(HEbig).save(
                    os.path.join(imcropdst, dstfn))
    Image.fromarray(DLrot.astype('uint32')).save(
                    os.path.join(dlcropdst, dstfn))
    Image.fromarray(nucrot).save(
                    os.path.join(nuccropdst, dstfn))

section N:  1 / 4


TypeError: Cannot handle this data type: (1, 1, 3), <u4

In [26]:
Image.fromarray(nucrot).save(
                os.path.join(nuccropdst, dstfn))

In [27]:
imtmp11=np.array(Image.open(r"\\fatherserverdw\kyuex\datadst\20220929\nuccrop\2022-06-10 18.30.09sec1.png"))
len(np.unique(imtmp11)),np.max(imtmp11)

(2790, 7670)